In [2]:
from absl import app
from absl import flags
import apache_beam as beam
from proto.stu3 import google_extensions_pb2
from proto.stu3 import resources_pb2
from py.google.fhir.labels import encounter
from py.google.fhir.labels import label


In [19]:
input_path = 'gs://healthedatalab/synthea/bundles/bundles*'
output_path = 'gs://healthedatalab/synthea/labels/label'

In [20]:
@beam.typehints.with_input_types(resources_pb2.Bundle)
@beam.typehints.with_output_types(google_extensions_pb2.EventLabel)
class LengthOfStayRangeLabelAt24HoursFn(beam.DoFn):
  """Converts Bundle into length of stay range at 24 hours label.
    Cohort: inpatient encounter that is longer than 24 hours
    Trigger point: 24 hours after admission
    Label: multi-label for length of stay ranges, see label.py for detail
  """

  def __init__(self, for_synthea=False):
    self._for_synthea = for_synthea

  def process(self, bundle):
    """Iterate through bundle and yield label.
    Args:
      bundle: input stu3.Bundle proto
    Yields:
      stu3.EventLabel proto.
    """
    patient = encounter.GetPatient(bundle)
    if patient is not None:
      # Cohort: inpatient encounter > 24 hours.
      for enc in encounter.Inpatient24HrEncounters(bundle, self._for_synthea):
        for one_label in label.LengthOfStayRangeAt24Hours(patient, enc):
          yield one_label



## Initialize pipeline variables

In [21]:
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.options.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import PipelineOptions

from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from apache_beam.metrics import Metrics
from apache_beam.metrics.metric import MetricsFilter

import apache_beam as beam
import re

options = PipelineOptions()
google_cloud_options = options.view_as(GoogleCloudOptions)
google_cloud_options.project = 'de-test-224618'
google_cloud_options.job_name = 'job1'
google_cloud_options.staging_location = 'gs://healthedatalab/staging'
google_cloud_options.temp_location = 'gs://healthedatalab/temp'
options.view_as(StandardOptions).runner = 'DirectRunner'

## Initalize the beam job

In [22]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
p = beam.Pipeline(options=options)

bundles = p | 'read' >> beam.io.ReadFromTFRecord(
    input_path, coder=beam.coders.ProtoCoder(resources_pb2.Bundle))
    
labels = bundles | 'BundleToLabel' >> beam.ParDo(
    LengthOfStayRangeLabelAt24HoursFn(for_synthea=True))
_ = labels | beam.io.WriteToTFRecord(
    output_path,
    coder=beam.coders.ProtoCoder(google_extensions_pb2.EventLabel),
    file_name_suffix='.tfrecords')

I0111 21:06:45.081870 140056621651712 gcsio.py:446] Starting the size estimation of the input
I0111 21:06:45.084059 140056621651712 client.py:614] Attempting refresh to obtain initial access_token
I0111 21:06:45.201682 140056621651712 gcsio.py:460] Finished listing 10 files in 0.119797945023 seconds.


## Run the transform

In [23]:
   p.run().wait_until_finish()

I0111 21:06:50.939930 140056621651712 fn_api_runner.py:844] ==================== <function annotate_downstream_side_inputs at 0x7f6132575a28> ====================
I0111 21:06:50.945723 140056621651712 fn_api_runner.py:844] ==================== <function fix_side_input_pcoll_coders at 0x7f6132575e60> ====================
I0111 21:06:50.949475 140056621651712 fn_api_runner.py:844] ==================== <function lift_combiners at 0x7f6132575cf8> ====================
I0111 21:06:50.951680 140056621651712 fn_api_runner.py:844] ==================== <function expand_gbk at 0x7f6132575f50> ====================
I0111 21:06:50.955533 140056621651712 fn_api_runner.py:844] ==================== <function sink_flattens at 0x7f6132575de8> ====================
I0111 21:06:50.957216 140056621651712 fn_api_runner.py:844] ==================== <function greedily_fuse at 0x7f6132575ed8> ====================
I0111 21:06:50.960843 140056621651712 fn_api_runner.py:844] ==================== <function sort_stag

I0111 21:07:17.198107 140056621651712 bundle_processor.py:303] finish <DataInputOperation WriteToTFRecord/Write/WriteImpl/GroupByKey/Read receivers=[ConsumerSet[WriteToTFRecord/Write/WriteImpl/GroupByKey/Read.out0, coder=WindowedValueCoder[TupleCoder[LengthPrefixCoder[FastPrimitivesCoder], IterableCoder[LengthPrefixCoder[FastPrimitivesCoder]]]], len(consumers)=1]]>
I0111 21:07:17.199670 140056621651712 bundle_processor.py:303] finish <DoOperation WriteToTFRecord/Write/WriteImpl/Extract output_tags=['out'], receivers=[ConsumerSet[WriteToTFRecord/Write/WriteImpl/Extract.out0, coder=WindowedValueCoder[LengthPrefixCoder[FastPrimitivesCoder]], len(consumers)=1]]>
I0111 21:07:17.201383 140056621651712 bundle_processor.py:303] finish <DataOutputOperation ref_PCollection_PCollection_11/Write >
I0111 21:07:17.207993 140056621651712 fn_api_runner.py:912] Running ((ref_PCollection_PCollection_3/Read)+(ref_AppliedPTransform_WriteToTFRecord/Write/WriteImpl/PreFinalize_19))+(ref_PCollection_PCollect

'DONE'

YAY!!